In [1]:
import os
import pandas as pd
import numpy as np
import fitsio
from pathlib import Path, PurePath
import cudf
from IPython.display import display

In [2]:
def create_lookup_8nb(nx, ny):
    """ Pre-compute the 8-connectivity lookup table. This will be shared across parallel workers.
    :param nx: number of columns in image array (number of pixels on horizontal axis)
    :param ny: number of rows in image array (number of pixels on vertical axis)
    :return:
    """
    # List of relative 2D coordinates for 8-neighbour connectivity, including origin pixel.
    coords_8nb = np.array([[0, 0], [-1, 0], [-1, -1], [0, -1], [1, -1], [1, 0], [1, 1], [0, 1], [-1, 1]])
    # Array of 2D coordinates for a 4096 x 4096 array. Matrix convention is kept. [rows, cols] = [y-axis, x-axis]
    coords_1d = np.arange(nx * ny)
    coordy, coordx = np.unravel_index(coords_1d, [ny, nx]) # also possible by raveling a meshgrid() output
    coords2d = np.array([coordy, coordx])
    # Create the array of 2D coordinates of 8-neighbours associated with each pixel.
    # pixel 0 has 8 neighbour + itself, pixel 1 has 8 neighbour + itself, etc...
    coords2d_8nb = coords2d[np.newaxis, ...] + coords_8nb[..., np.newaxis]
    # Handle off-edges coordinates by clipping to the edges, operation done in-place. Here, square detector assumed.
    # to per-axis clipping if that ever changes for another instrument.
    np.clip(coords2d_8nb, 0, nx-1, out=coords2d_8nb)
    # Convert to 1D coordinates.
    lookup_coords = np.array([coords2d_8nb[i, 0, :] * nx + coords2d_8nb[i, 1, :] for i in range(len(coords_8nb))],
                         dtype='int32', order='C').T
    return lookup_coords

### Load the dataframes of file paths and their timestamp. 

In [3]:
spikes_df = pd.read_parquet(os.path.join(os.environ['SPIKESDATA'], 'spikes_df_2010.parquet'), engine='pyarrow')
spikes_df.set_index(['GroupNumber'], inplace=True)
path_Series = spikes_df['Path']

In [4]:
tintervals = pd.interval_range(start=pd.Timestamp('2010-05-13 00:00:00', tz='UTC'),
                                   end=pd.Timestamp('2010-05-16 00:00:00', tz='UTC'),
                                   freq='D', closed='left')

### Get the file paths of the 1st group in that time interval. There are 7 files per group

In [5]:
tint = tintervals[0]
groups = spikes_df.loc[(spikes_df['Time'] >= tint.left) & (spikes_df['Time'] < tint.right)].index.unique()
group_n = groups[0]
fpaths = path_Series[group_n]
fpaths

0    2010/05/13/2010-05-13T00:00:02.09Z_0193.spikes...
0    2010/05/13/2010-05-13T00:00:03.57Z_0094.spikes...
0    2010/05/13/2010-05-13T00:00:05.07Z_0335.spikes...
0    2010/05/13/2010-05-13T00:00:06.58Z_0171.spikes...
0    2010/05/13/2010-05-13T00:00:08.08Z_0211.spikes...
0    2010/05/13/2010-05-13T00:00:09.58Z_0304.spikes...
0    2010/05/13/2010-05-13T00:00:11.08Z_0131.spikes...
Name: Path, dtype: object

### load these fits files in RAM and send to GPU

In [6]:
# %%timeit 3.99 ms ± 257 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
spikes_list = [fitsio.read(os.path.join(os.environ['SPIKESDATA'], f)) for f in fpaths]
for spikes in spikes_list:
    print(len(spikes[0,:]))
# Note: the size of the series of coordinates varies between files. They are independent measurements of "spikes" in each CCD <-> wavelength

8486
30356
36549
7993
13781
26443
27576


In [7]:
# To GPU: list of CUDF Series containing only the coordinates from the data loaded in each file
# 7 CUDF Series cooresponding to the spikes coordinates measured in the 7 wavelengths (wav0, wav1, ... wav6)
cuseries_ = [cudf.Series(spikes[0,:], name=f'wav{w}') for w, spikes in enumerate(spikes_list)]

## Create lookup table and send to GPU

In [8]:
index_8nb = create_lookup_8nb(4096, 4096)

In [9]:
pd_temp = pd.DataFrame(index_8nb, index=index_8nb[:,0])
cuindex_8nb = cudf.DataFrame.from_pandas(pd_temp)
display(pd_temp.head())

,0,1,2,3,4,5,6,7,8
0,0,0,0,0,4096,4096,4097,1,1
1,1,1,0,0,4096,4097,4098,2,2
2,2,2,1,1,4097,4098,4099,3,3
3,3,3,2,2,4098,4099,4100,4,4
4,4,4,3,3,4099,4100,4101,5,5


## Define the "needles and the haystack" 

In [10]:
# Spikes coordinates at a given wavelength (starting with 1st one, i.e, index 0)
spikes = cuseries_[0]
# needles: series of spikes coordinates in another wavelength from which occurences will be searched in the above haystack
needles_pixels = cuseries_[1]
# Haystack variable: spikes coordinates at one wavelength with the coordinates of their 8 nearest neighbours. 
# Haystack is assigned True wherever elements of needles_pixels are found. 
haystack_pixels = cuindex_8nb.iloc[spikes, :]
# Show the needles and haystack
display(haystack_pixels.head())
display(needles_pixels.head())

,0,1,2,3,4,5,6,7,8
9362,9362,5266,5265,9361,13457,13458,13459,9363,5267
9706,9706,5610,5609,9705,13801,13802,13803,9707,5611
10170,10170,6074,6073,10169,14265,14266,14267,10171,6075
10726,10726,6630,6629,10725,14821,14822,14823,10727,6631
13014,13014,8918,8917,13013,17109,17110,17111,13015,8919


0     9412
1    10636
2    11175
3    11426
4    12900
Name: wav1, dtype: int32

In [11]:
# For comparison with numpy (CPU) 
haystack_pixels_np = index_8nb[spikes_list[0][0,:], :]
needles_np = spikes_list[1][0,:]
display(haystack_pixels_np[0:5, :])

array([[ 9362,  5266,  5265,  9361, 13457, 13458, 13459,  9363,  5267],
       [ 9706,  5610,  5609,  9705, 13801, 13802, 13803,  9707,  5611],
       [10170,  6074,  6073, 10169, 14265, 14266, 14267, 10171,  6075],
       [10726,  6630,  6629, 10725, 14821, 14822, 14823, 10727,  6631],
       [13014,  8918,  8917, 13013, 17109, 17110, 17111, 13015,  8919]],
      dtype=int32)

### processing with Numpy / CPU

In [12]:
%timeit np_H0_1 = np.isin(haystack_pixels_np, needles_np).any(axis=1)

4.35 ms ± 77.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### processing with CUDF / GPU
Note: with CUDF, loop *isin()* through the 9 series, as *isin()* is only implemented for Series and not Dataframes contrary to Numpy above. 

In [14]:
%timeit cudf_H0_1 = cudf.concat([haystack_pixels[i].isin(needles_pixels) for i in range(9)], axis=1).any(axis=1)

130 ms ± 1.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Maybe the list comprehension could be processed concurently by the GPU too?
Also, that's just for wavelength 0 as haystack, to be searched for 6 other lists of needles from the 6 other wavelengths. 
Each of the 6 other wavelength will also become the haystack and so that all process will repeat also 7 times. This is to account for all possible combinations of 8-connectivity neighbourhoods. 